In [1]:
import tensorflow as tf

In [2]:
import os
os.chdir('..')  # Run this only once

In [144]:
from my_io import Dataset
dataset = Dataset('fraction')

prefix is now tmp/0-0


In [146]:
from sklearn.model_selection import ShuffleSplit

In [148]:
triplets = []
with open('/Users/jin/Sites/mangaki/data/ratings-ml.csv') as f:
    reader = csv.reader(f)
    for user_id, work_id, rating in reader:
        triplets.append([int(user_id), int(work_id), 1 if float(rating) >= 2.5 else 0])

In [154]:
k_fold = ShuffleSplit(n_splits=5)

In [157]:
for i_train, i_test in k_fold.split(triplets):
    pass

In [160]:
train = np.array(triplets)[i_train]
test = np.array(triplets)[i_test]

In [161]:
import numpy as np
#X = tf.constant(np.array(dataset.data).astype(np.float32))

In [12]:
import csv
import random

def get_train_test(data):
    n = len(data)
    m = len(data[0])
    train = []
    test = []
    for i in range(n):
        for j in range(m):
            if random.random() < 0.9:
                train.append((i, j, int(data[i][j])))
            else:
                test.append((i, j, int(data[i][j])))
    return train, test

In [13]:
def base(i, d):
    v = [0] * d
    v[i] = 1
    return v

In [7]:
train, test = get_train_test(dataset.data)

In [166]:
NB_USERS = 1 + max(max(train[:, 0]), max(test[:, 0]))
NB_WORKS = 1 + max(max(train[:, 1]), max(test[:, 1]))

def make_hot(triplets):
    data = []
    for i, j, v in triplets:
        data.append((base(i, NB_USERS), base(j, NB_WORKS), v))
    random.shuffle(data)
    return data

In [167]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield len(l[i:i + n]), l[i:i + n]

In [168]:
def get_batches(data):
    return list(map(np.array, zip(*data)))

In [169]:
data = make_hot(train)

In [182]:
N = NB_USERS
R = 20
M = NB_WORKS

In [183]:
U = tf.Variable(tf.random_normal([N, R]), name='U')
V = tf.Variable(tf.random_normal([R, M]), name='V')
bias = tf.Variable(tf.random_normal([M, 1]), name='bias')

In [184]:
BATCH_SIZE = 100
E = tf.placeholder(tf.float32, shape=(None, N))
F = tf.placeholder(tf.float32, shape=(None, M))
T = tf.placeholder(tf.float32, shape=None)

In [185]:
pred_without_bias = tf.reduce_sum(tf.matmul(E, U) * tf.matmul(F, tf.transpose(V)), axis=1)
pred = pred_without_bias + tf.reduce_sum(tf.matmul(F, bias), axis=1)

In [186]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=T, logits=pred))
regularized_loss = (loss + 0.01 * tf.reduce_mean(tf.square(U))
                         + 0.01 * tf.reduce_mean(tf.abs(V))
                         + 0.01 * tf.reduce_mean(tf.square(bias)))

In [187]:
optimizer = tf.train.AdamOptimizer(0.01)
train_step = optimizer.minimize(regularized_loss, var_list=[U, V, bias])
#train_V_step = optimizer.minimize(negative_log_likelihood, var_list=[V])

In [188]:
init_op = tf.global_variables_initializer()

In [189]:
sess = tf.InteractiveSession()
sess.run(init_op)

In [180]:
test_data = make_hot(test)

In [ ]:
for iteration in range(20):
    for size, chunk in chunks(data, BATCH_SIZE):
        if size < BATCH_SIZE:
            break
        eb, fb, vb = get_batches(chunk)
        sess.run(train_step, feed_dict={E: eb, F: fb, T: vb})
        #print(sess.run(loss, feed_dict={E: eb, F: fb, T: vb}))

    if iteration % 1 == 0:
        eb, fb, vb = get_batches(test_data)
        print(iteration, sess.run(loss, feed_dict={E: eb, F: fb, T: vb}))        

La bonne nouvelle, c'est qu'on obtient une log-likelihood de -3606 en 20 itérations alors que je viens de check, le package R converge au bout de 210 itérations vers une log-likelihood de -4454.

Reste à savoir si ça généralise bien. (Le package R lui suppose un prior bayésien sur les compétences des gens.)

In [15]:
sess.run(tf.transpose(tf.abs(V)))

array([[  4.92225617e-01,   7.57199168e-01,   2.99945354e-01,
          1.26648080e+00,   8.36898029e-01,   7.46591032e-01,
          5.75143278e-01,   5.81251383e-01],
       [  5.69952667e-01,   9.71436977e-01,   4.82483119e-01,
          1.65013766e+00,   1.00464725e+00,   8.38633776e-01,
          3.42653811e-01,   3.41077089e-01],
       [  5.75150549e-01,   4.04434323e-01,   4.55957234e-01,
          7.02938914e-01,   7.80022323e-01,   7.22231090e-01,
          7.50544131e-01,   6.22578025e-01],
       [  6.33334458e-01,   4.12763625e-01,   4.64858294e-01,
          2.71402687e-01,   4.97841388e-01,   1.91181988e-01,
          8.77478123e-02,   7.52362609e-01],
       [  1.95949644e-01,   3.03780377e-01,   2.94619501e-01,
          3.12623233e-01,   7.60286033e-01,   2.62267590e-01,
          1.30510494e-01,   2.11985722e-01],
       [  5.13305902e-01,   6.01617277e-01,   1.24468535e-01,
          6.20978296e-01,   3.03121835e-01,   1.04155182e-03,
          4.01907325e-01,   4.9

In [27]:
tf_mask

<tf.Tensor 'Const_7:0' shape=(536, 20) dtype=float32>